# Neural networks

In [13]:
import gzip, pickle, time, torch, os, sys
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Pytorch를 활용한 Network 정의

### 간단한 Fully-connected neural network 정의

In [3]:
class FCNN(nn.Module):
    # 모델 정의
    def __init__(self, 
                 num_feat,
                 num_output,
                 num_node = 32,
                 num_layer = 4,
                 dropout_rate = 0,
                 batch_norm = False,
        ):
        # nn.Module 초기화
        super(FCNN, self).__init__()
        
        # embedding layer 정의
        self.embed = nn.Sequential(
            nn.Linear(num_feat, num_node), # Linear layer
            nn.ReLU(), # activation function
        )
        
        # hidden layer 정의
        self.hidden = nn.ModuleList()
        for _ in range(num_layer):
            hidden_layer = [] # List 형태로 만든 후 순차적으로 layer 요소 추가
            hidden_layer.append(nn.Linear(num_node, num_node)) 
            hidden_layer.append(nn.ReLU())

            # Layer normalization, gradient가 발산/수렴하는 것을 방지해서 훈련 효율을 높임
            # 둘 중 하나만 사용하면 됨
            if dropout_rate != 0: # Dropout 정의
                hidden_layer.append(nn.Dropout(dropout_rate))
            if batch_norm: # batch normalization 정의
                hidden_layer.append(nn.BatchNorm1d(num_node))
            self.hidden.append(nn.Sequential(*hidden_layer))

        self.output = nn.Sequential(
            nn.Linear(num_node, num_output),
        )
    
    # 순방향 함수 정의
    # input x가 들어와서 어떤 연산을 거쳐 output이 될 지 정의하는 함수
    def forward(self, x):
        # embedding layer 통과
        h = self.embed(x)
        # 각 layer를 순차적으로 통과
        for layer in self.hidden:
            h = layer(h)
        # output layer 통과
        out = self.output(h)
        return out

### Convolution neural network 정의

In [4]:
class CNN2D(nn.Module):
    # 모델 정의
    def __init__(self, 
                 in_channel,
                 num_output,
                 out_channel = 32,
                 kernel_size = 4,
                 stride = 1,
                 padding = 0,
                 dilation = 1,
                 num_layer = 3,
                 dropout_rate = 0,
                 batch_norm = False,
        ):
        # nn.Module 초기화
        super(CNN2D, self).__init__()
        self.embed = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding, dilation),
            nn.LeakyReLU(0.1)
        )
        # hidden layer 정의
        self.hidden = nn.ModuleList()
        for _ in range(num_layer):
            hidden_layer = [] # List 형태로 만든 후 순차적으로 layer 요소 추가
            hidden_layer.append(nn.Conv2d(out_channel, out_channel, kernel_size, stride, padding, dilation)) 
            hidden_layer.append(nn.LeakyReLU(0.1))

            # Layer normalization, gradient가 발산/수렴하는 것을 방지해서 훈련 효율을 높임
            # 둘 중 하나만 사용하면 됨
            if dropout_rate != 0: # Dropout 정의
                hidden_layer.append(nn.Dropout(dropout_rate))
            if batch_norm: # batch normalization 정의
                hidden_layer.append(nn.BatchNorm2d(out_channel))
            hidden_layer.append(nn.MaxPool2d(2))
            self.hidden.append(nn.Sequential(*hidden_layer))

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, num_output)
        )
    
    # 순방향 함수 정의
    # input x가 들어와서 어떤 연산을 거쳐 output이 될 지 정의하는 함수
    def forward(self, x):
        # embedding layer 통과
        h = self.embed(x)
#        print(h.shape)
        # 각 layer를 순차적으로 통과
        for layer in self.hidden:
            h = layer(h)
#            print(h.shape)
        # output layer 통과
        out = self.output(h)
#        print(out.shape)
        return out

### MNIST dataset 테스트

In [14]:
with gzip.open('../data/mnist.pkl.gzip','rb') as f:
    mnist = pickle.load(f)

x_mnist_train = mnist['train']['x']
y_mnist_train = mnist['train']['y']
x_mnist_test = mnist['test']['x']
y_mnist_test = mnist['test']['y']

x_train = torch.from_numpy(x_mnist_train).short() # 16-bit integer
x_train.to(device)
# long code ...

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int16)

In [36]:
class FlatData(Dataset):
    def __init__(self, x, y, dtype=torch.float, device='cpu'):
        # Dataset 초기화
        super(FlatData, self).__init__()
        # Input이 Tensor일 경우와 Tensor가 아닐 경우
        if isinstance(x, torch.Tensor):
            self._x = x.type(dtype).to(device)
        else:
            self._x = torch.tensor(x, dtype=dtype).to(device)
        
        # Label에 대해서도 동일한 작업 수행
        if isinstance(y, torch.Tensor):
            self._y = y.type(dtype).to(device)
        else:
            self._y = torch.tensor(y, dtype=dtype).to(device)

    def __len__(self):
        return self._x.shape[0]
    
    def __getitem__(self, i):
        return self._x[i], self._y[i]

In [40]:
train_dataset = FlatData(x_mnist_train, y_mnist_train, dtype=torch.short)
test_dataset = FlatData(x_mnist_test, y_mnist_test, dtype=torch.short)

DataLoader(train_dataset)
DataLoader(test_dataset)

(tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,  13,  25, 100, 122,   7,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0